# Approaching (Almost) Any NLP Problem on Kaggle

In this post I'll talk about approaching natural language processing problems on Kaggle. As an example, we will use the data from this competition. We will create a very basic first model first and then improve it using different other features. We will also see how deep neural networks can be used and end this post with some ideas about ensembling in general.

### This covers:
- tfidf 
- count features
- logistic regression
- naive bayes
- svm
- xgboost
- grid search
- word vectors
- LSTM
- GRU
- Ensembling

*NOTE*: This notebook is not meant for achieving a very high score on the Leaderboard for this dataset. However, if you follow it properly, you can get a very high score with some tuning. ;)

So, without wasting any time, let's start with importing some important python modules that I'll be using.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

2022-11-19 14:17:42.312578: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-19 14:17:42.312623: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Let's load the datasets

In [2]:
train = pd.read_csv('/data/elastic-notebook/data/spooky/train.csv')
test = pd.read_csv('/data/elastic-notebook/data/spooky/test.csv')
sample = pd.read_csv('/data/elastic-notebook/data/spooky/sample_submission.csv')

A quick look at the data

In [3]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [4]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [5]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


The problem requires us to predict the author, i.e. EAP, HPL and MWS given the text. In simpler words, text classification with 3 different classes.

For this particular problem, Kaggle has specified multi-class log-loss as evaluation metric. This is implemented in the follow way (taken from: https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne/util.py)

In [6]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

We use the LabelEncoder from scikit-learn to convert text labels to integers, 0, 1 2

In [7]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

Before going further it is important that we split the data into training and validation sets. We can do it using `train_test_split` from the `model_selection` module of scikit-learn.

In [8]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [9]:
print (xtrain.shape)
print (xvalid.shape)

(17621,)
(1958,)


## Building Basic Models

Let's start building our very first model. 

Our very first model is a simple TF-IDF (Term Frequency - Inverse Document Frequency) followed by a simple Logistic Regression.

In [10]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [11]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.572 


/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


And there we go. We have our first model with a multiclass logloss of 0.626.

But we are greedy and want a better score. Lets look at the same model with a different data.

Instead of using TF-IDF, we can also use word counts as features. This can be done easily using CountVectorizer from scikit-learn.

In [12]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [13]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.527 


/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Aaaaanddddddd Wallah! We just improved our first model by 0.1!!!

Next, let's try a very simple model which was quite famous in ancient times - Naive Bayes.

Let's see what happens when we use naive bayes on these two datasets:

In [14]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.578 


Good performance! But the logistic regression on counts is still better! What happens when we use this model on counts data instead?

In [15]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.485 


Whoa! Seems like old stuff still works good!!!! One more ancient algorithms in the list is SVMs. Some people "love" SVMs. So, we must try SVM on this dataset.

Since SVMs take a lot of time, we will reduce the number of features from the TF-IDF using Singular Value Decomposition before applying SVM. 

Also, note that before applying SVMs, we *must* standardize the data.

In [16]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

Now it's time to apply SVM. After running the following cell, feel free to go for a walk or talk to your girlfriend/boyfriend. :P

In [17]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.729 


Oops! time to get up! Looks like SVM doesn't perform well on this data...! 

Before moving further, lets apply the most popular algorithm on Kaggle: xgboost!

In [18]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.780 


In [19]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.773 


In [20]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.762 


In [21]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.791 


Seems like no luck with XGBoost! But that is not correct. I haven't done any hyperparameter optimizations yet. And since I'm lazy, I'll just tell you how to do it and you can do it on your own! ;). This will be discussed in the next section:


## Grid Search

Its a technique for hyperparameter optimization. Not so effective but can give good results if you know the grid you want to use. I specify the parameters that should usually be used in this post: http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/ Please keep in mind that these are the parameters I usually use. There are many other methods of hyperparameter optimization which may or may not be as effective.

In this section, I'll talk about grid search using logistic regression. 

Before starting with grid search we need to create a scoring function. This is accomplished using the `make_scorer` function of scikit-learn.


In [22]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

Next we need a pipeline. For demonstration here, i'll be using a pipeline consisting of SVD, scaling and then logistic regression. Its better to understand with more modules in pipeline than just one ;)

In [23]:
# Initialize SVD
svd = TruncatedSVD()
    
# Initialize the standard scaler 
scl = preprocessing.StandardScaler()

# We will use logistic regression here..
lr_model = LogisticRegression()

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

Next we need a grid of parameters:

In [24]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

So, for SVD we evaluate 120 and 180 components and for logistic regression we evaluate three different values of C with l1 and l2 penalty. We can now start grid search on these parameters.

In [26]:
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/linear_m

Best score: -0.739
Best parameters set:
	lr__C: 10
	lr__penalty: 'l2'
	svd__n_components: 180


The score comes similar to what we had for SVM. This technique can be used to finetune xgboost or even multinomial naive bayes as below. We will use the tfidf data here:

In [28]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best score: -0.492
Best parameters set:
	nb__alpha: 0.1


This is an improvement of 8% over the original naive bayes score!

In NLP problems, it's customary to look at word vectors. Word vectors give a lot of insights about the data. Let's dive into that.

## Word Vectors

Without going into too much details, I would explain how to create sentence vectors and how can we use them to create a machine learning model on top of it. I am a fan of GloVe vectors, word2vec and fasttext. In this post, I'll be using the GloVe vectors. You can download the GloVe vectors from here `http://www-nlp.stanford.edu/data/glove.840B.300d.zip`

In [35]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('/data/elastic-notebook/data/glove.840B.300d/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.strip().split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

1356179it [01:07, 19892.30it/s]

[CV 1/2; 3/12] START lr__C=0.1, lr__penalty=l2, svd__n_components=120...........
[CV 1/2; 3/12] END lr__C=0.1, lr__penalty=l2, svd__n_components=120;, score=-0.786 total time=   0.8s
[CV 2/2; 1/12] START lr__C=0.1, lr__penalty=l1, svd__n_components=120...........
[CV 2/2; 1/12] END lr__C=0.1, lr__penalty=l1, svd__n_components=120;, score=nan total time=   0.8s
[CV 1/2; 1/12] START lr__C=0.1, lr__penalty=l1, svd__n_components=120...........
[CV 1/2; 1/12] END lr__C=0.1, lr__penalty=l1, svd__n_components=120;, score=nan total time=   0.8s


1360155it [01:07, 19851.98it/s]

[CV 1/2; 9/12] START lr__C=10, lr__penalty=l1, svd__n_components=120............
[CV 1/2; 9/12] END lr__C=10, lr__penalty=l1, svd__n_components=120;, score=nan total time=   0.8s
[CV 2/2; 9/12] START lr__C=10, lr__penalty=l1, svd__n_components=120............
[CV 2/2; 9/12] END lr__C=10, lr__penalty=l1, svd__n_components=120;, score=nan total time=   0.9s
[CV 2/2; 3/12] START lr__C=0.1, lr__penalty=l2, svd__n_components=120...........
[CV 2/2; 3/12] END lr__C=0.1, lr__penalty=l2, svd__n_components=120;, score=-0.773 total time=   1.0s


1366142it [01:07, 19915.25it/s]

...

[CV 2/2; 10/12] START lr__C=10, lr__penalty=l1, svd__n_components=180...........
[CV 2/2; 10/12] END lr__C=10, lr__penalty=l1, svd__n_components=180;, score=nan total time=   1.5s
[CV 2/2; 4/12] START lr__C=0.1, lr__penalty=l2, svd__n_components=180...........
[CV 2/2; 4/12] END lr__C=0.1, lr__penalty=l2, svd__n_components=180;, score=-0.736 total time=   1.5s
[CV 1/2; 8/12] START lr__C=1.0, lr__penalty=l2, svd__n_components=180...........
[CV 1/2; 8/12] END lr__C=1.0, lr__penalty=l2, svd__n_components=180;, score=-0.745 total time=   1.6s


1392164it [01:08, 19987.01it/s]

[CV 2/2; 8/12] START lr__C=1.0, lr__penalty=l2, svd__n_components=180...........
[CV 2/2; 8/12] END lr__C=1.0, lr__penalty=l2, svd__n_components=180;, score=-0.736 total time=   1.6s
[CV 2/2; 12/12] START lr__C=10, lr__penalty=l2, svd__n_components=180...........
[CV 2/2; 12/12] END lr__C=10, lr__penalty=l2, svd__n_components=180;, score=-0.738 total time=   1.7s


2196018it [01:48, 20217.47it/s]

Found 2196008 word vectors.


In [40]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

[CV 1/2; 6/6] START nb__alpha=100...............................................
[CV 1/2; 6/6] END ...............nb__alpha=100;, score=-1.067 total time=   0.0s
[CV 1/2; 5/6] START nb__alpha=10................................................
[CV 1/2; 5/6] END ................nb__alpha=10;, score=-0.950 total time=   0.0s
[CV 2/2; 6/6] START nb__alpha=100...............................................
[CV 2/2; 6/6] END ...............nb__alpha=100;, score=-1.067 total time=   0.0s
[CV 1/2; 1/6] START nb__alpha=0.001.............................................
[CV 1/2; 1/6] END .............nb__alpha=0.001;, score=-0.620 total time=   0.0s


In [41]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]

[CV 1/2; 2/6] START nb__alpha=0.01..............................................
[CV 1/2; 2/6] END ..............nb__alpha=0.01;, score=-0.511 total time=   0.0s




  0%|                                                                                              | 0/17621 [00:00<?, ?it/s]

  2%|█▉                                                                                | 412/17621 [00:00<00:04, 4117.03it/s]

[CV 1/2; 3/6] START nb__alpha=0.1...............................................
[CV 1/2; 3/6] END ...............nb__alpha=0.1;, score=-0.489 total time=   0.0s
[CV 2/2; 2/6] START nb__alpha=0.01..............................................
[CV 2/2; 2/6] END ..............nb__alpha=0.01;, score=-0.523 total time=   0.0s
[CV 2/2; 1/6] START nb__alpha=0.001.............................................
[CV 2/2; 1/6] END .............nb__alpha=0.001;, score=-0.641 total time=   0.0s




  5%|████▏                                                                             | 904/17621 [00:00<00:03, 4588.86it/s]

  8%|██████▌                                                                          | 1425/17621 [00:00<00:03, 4870.01it/s]

 11%|█████████                                                                        | 1962/17621 [00:00<00:03, 5063.64it/s]

[CV 2/2; 3/6] START nb__alpha=0.1...............................................
[CV 2/2; 3/6] END ...............nb__alpha=0.1;, score=-0.495 total time=   0.0s
[CV 1/2; 4/6] START nb__alpha=1.................................................
[CV 1/2; 4/6] END .................nb__alpha=1;, score=-0.663 total time=   0.0s
[CV 2/2; 4/6] START nb__alpha=1.................................................
[CV 2/2; 4/6] END .................nb__alpha=1;, score=-0.666 total time=   0.0s




 14%|███████████▎                                                                     | 2474/17621 [00:00<00:02, 5080.08it/s]

 17%|█████████████▊                                                                   | 3009/17621 [00:00<00:02, 5171.16it/s]

 20%|████████████████▎                                                                | 3556/17621 [00:00<00:02, 5268.65it/s]

 23%|██████████████████▊                                                              | 4097/17621 [00:00<00:02, 5312.44it/s]

 26%|█████████████████████▎                                                           | 4647/17621 [00:00<00:02, 5370.62it/s]

 29%|███████████████████████▊                                                         | 5188/17621 [00:01<00:02, 5381.54it/s]

 33%|██████████████████████████▎                                                      | 5727/17621 [00:01<00:02, 5320.47it/s]

 36%|████████████████████████████▊                                                    | 6260/17621 [00:01<00:

In [42]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

Let's see the performance of xgboost on glove features:

In [43]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

[14:47:31] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

logloss: 0.716 


In [44]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

[14:47:58] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

logloss: 0.683 


we see that a simple tuning of parameters can improve xgboost score on GloVe features! Believe me you can squeeze a lot more from it.

## Deep Learning

But this is an era of deep learning! We cant live without training a few neural networks. Here, we will train LSTM and a simple dense network on the GloVe features. Let's start with the dense network first:

In [45]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [46]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [47]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

2022-11-19 14:48:48.009825: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-19 14:48:48.009869: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-19 14:48:48.009891: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (yongjoo-serv-01): /proc/driver/nvidia/version does not exist
2022-11-19 14:48:48.010179: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
  0%|                                                                                              | 0/1762

In [48]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Epoch 1/5
276/276 [==============================] - 2s 5ms/step - loss: 0.9153 - val_loss: 0.7081
Epoch 2/5
276/276 [==============================] - 1s 5ms/step - loss: 0.6958 - val_loss: 0.6761
Epoch 3/5
276/276 [==============================] - 1s 5ms/step - loss: 0.6417 - val_loss: 0.6745
Epoch 4/5
276/276 [==============================] - 1s 5ms/step - loss: 0.5976 - val_loss: 0.6733
Epoch 5/5
276/276 [==============================] - 1s 5ms/step - loss: 0.5606 - val_loss: 0.6686


You need to keep on tuning the parameters of the neural network, add more layers, increase dropout to get better results. Here, I'm just showing that its fast to implement and run and gets better result than xgboost without any optimization :)

To move further, i.e. with LSTMs we need to tokenize the text data

In [49]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [50]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████████████████████████████████████████████████████████████████████████| 25943/25943 [00:00<00:00, 273234.99it/s]


In [51]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [52]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Epoch 1/100
35/35 [==============================] - 13s 289ms/step - loss: 1.0653 - val_loss: 0.9358
Epoch 2/100
35/35 [==============================] - 10s 282ms/step - loss: 0.9023 - val_loss: 0.7690
Epoch 3/100
35/35 [==============================] - 9s 251ms/step - loss: 0.8180 - val_loss: 0.7097
Epoch 4/100
35/35 [==============================] - 10s 273ms/step - loss: 0.7817 - val_loss: 0.7146
Epoch 5/100
35/35 [==============================] - 10s 273ms/step - loss: 0.7618 - val_loss: 0.6600
Epoch 6/100
35/35 [==============================] - 9s 263ms/step - loss: 0.7422 - val_loss: 0.6566
Epoch 7/100
35/35 [==============================] - 9s 248ms/step - loss: 0.7244 - val_loss: 0.6654
Epoch 8/100
35/35 [==============================] - 9s 243ms/step - loss: 0.7184 - val_loss: 0.6379
Epoch 9/100
35/35 [==============================] - 9s 244ms/step - loss: 0.6875 - val_loss: 0.6232
Epoch 10/100
35/35 [==============================] - 9s 243ms/step - loss: 0.6742 - va

We see that the score is now less than 0.5. I ran it for many epochs without stopping at the best but you can use early stopping to stop at the best iteration. How do I use early stopping?

well, pretty easy. let's compile the model again:

In [53]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 24s 611ms/step - loss: 1.0359 - val_loss: 0.8396
Epoch 2/100
35/35 [==============================] - 21s 601ms/step - loss: 0.8567 - val_loss: 0.7433
Epoch 3/100
35/35 [==============================] - 21s 589ms/step - loss: 0.8085 - val_loss: 0.7192
Epoch 4/100
35/35 [==============================] - 21s 612ms/step - loss: 0.7805 - val_loss: 0.7052
Epoch 5/100
35/35 [==============================] - 21s 591ms/step - loss: 0.7658 - val_loss: 0.6769
Epoch 6/100
35/35 [==============================] - 20s 577ms/step - loss: 0.7379 - val_loss: 0.6783
Epoch 7/100
35/35 [==============================] - 20s 579ms/step - loss: 0.7065 - val_loss: 0.6379
Epoch 8/100
35/35 [==============================] - 20s 582ms/step - loss: 0.6955 - val_loss: 0.6233
Epoch 9/100
35/35 [==============================] - 20s 573ms/step - loss: 0.6639 - val_loss: 0.6017
Epoch 10/100
35/35 [==============================] - 20s 573ms/step - loss: 0.634

One question could be: why do i use so much dropout? Well, fit the model with no or little dropout and you will that it starts to overfit :)

Let's see if Bi-directional LSTM can give us better results. Its a piece of cake to do it with Keras :)

In [54]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 36s 918ms/step - loss: 1.0256 - val_loss: 0.8178
Epoch 2/100
35/35 [==============================] - 31s 900ms/step - loss: 0.8500 - val_loss: 0.7567
Epoch 3/100
35/35 [==============================] - 32s 904ms/step - loss: 0.8104 - val_loss: 0.7492
Epoch 4/100
35/35 [==============================] - 31s 896ms/step - loss: 0.7823 - val_loss: 0.7098
Epoch 5/100
35/35 [==============================] - 31s 895ms/step - loss: 0.7687 - val_loss: 0.6929
Epoch 6/100
35/35 [==============================] - 31s 889ms/step - loss: 0.7374 - val_loss: 0.6566
Epoch 7/100
35/35 [==============================] - 31s 892ms/step - loss: 0.7160 - val_loss: 0.6316
Epoch 8/100
35/35 [==============================] - 31s 891ms/step - loss: 0.6909 - val_loss: 0.6418
Epoch 9/100
35/35 [==============================] - 31s 892ms/step - loss: 0.6546 - val_loss: 0.6036
Epoch 10/100
35/35 [==============================] - 32s 911ms/step - loss: 0.636

Pretty close! Lets try two layers of GRU:

In [55]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 39s 1s/step - loss: 1.0605 - val_loss: 0.9116
Epoch 2/100
35/35 [==============================] - 36s 1s/step - loss: 0.9143 - val_loss: 0.8079
Epoch 3/100
35/35 [==============================] - 36s 1s/step - loss: 0.8362 - val_loss: 0.7641
Epoch 4/100
35/35 [==============================] - 36s 1s/step - loss: 0.7964 - val_loss: 0.7070
Epoch 5/100
35/35 [==============================] - 36s 1s/step - loss: 0.7624 - val_loss: 0.6760
Epoch 6/100
35/35 [==============================] - 36s 1s/step - loss: 0.7429 - val_loss: 0.6528
Epoch 7/100
35/35 [==============================] - 36s 1s/step - loss: 0.7115 - val_loss: 0.6317
Epoch 8/100
35/35 [==============================] - 36s 1s/step - loss: 0.6851 - val_loss: 0.6079
Epoch 9/100
35/35 [==============================] - 36s 1s/step - loss: 0.6684 - val_loss: 0.5929
Epoch 10/100
35/35 [==============================] - 35s 1s/step - loss: 0.6281 - val_loss: 0.5672
Epoch 11/

Nice! Much better than what we had previously! Keep optimizing and the performance will keep improving.
Worth trying: stemming and lemmatization. This is something I'm skipping for now.

In the Kaggle world, to get a top score you should have an ensemble of models. Let's check a little bit of ensembling!


## Ensembling

Few months back I made a simple ensembler but I didn't have time to develop it fully. It can be found here: https://github.com/abhishekkrthakur/pysembler . I'm going to use some part of it here:

In [56]:
# this is the main ensembling class. how to use it is in the next cell!
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
import os
import sys
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format="[%(asctime)s] %(levelname)s %(message)s",
    datefmt="%H:%M:%S", stream=sys.stdout)
logger = logging.getLogger(__name__)


class Ensembler(object):
    def __init__(self, model_dict, num_folds=3, task_type='classification', optimize=roc_auc_score,
                 lower_is_better=False, save_path=None):
        """
        Ensembler init function
        :param model_dict: model dictionary, see README for its format
        :param num_folds: the number of folds for ensembling
        :param task_type: classification or regression
        :param optimize: the function to optimize for, e.g. AUC, logloss, etc. Must have two arguments y_test and y_pred
        :param lower_is_better: is lower value of optimization function better or higher
        :param save_path: path to which model pickles will be dumped to along with generated predictions, or None
        """

        self.model_dict = model_dict
        self.levels = len(self.model_dict)
        self.num_folds = num_folds
        self.task_type = task_type
        self.optimize = optimize
        self.lower_is_better = lower_is_better
        self.save_path = save_path

        self.training_data = None
        self.test_data = None
        self.y = None
        self.lbl_enc = None
        self.y_enc = None
        self.train_prediction_dict = None
        self.test_prediction_dict = None
        self.num_classes = None

    def fit(self, training_data, y, lentrain):
        """
        :param training_data: training data in tabular format
        :param y: binary, multi-class or regression
        :return: chain of models to be used in prediction
        """

        self.training_data = training_data
        self.y = y

        if self.task_type == 'classification':
            self.num_classes = len(np.unique(self.y))
            logger.info("Found %d classes", self.num_classes)
            self.lbl_enc = LabelEncoder()
            self.y_enc = self.lbl_enc.fit_transform(self.y)
            kf = StratifiedKFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, self.num_classes)
        else:
            self.num_classes = -1
            self.y_enc = self.y
            kf = KFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, 1)

        self.train_prediction_dict = {}
        for level in range(self.levels):
            self.train_prediction_dict[level] = np.zeros((train_prediction_shape[0],
                                                          train_prediction_shape[1] * len(self.model_dict[level])))

        for level in range(self.levels):

            if level == 0:
                temp_train = self.training_data
            else:
                temp_train = self.train_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):
                validation_scores = []
                foldnum = 1
                for train_index, valid_index in kf.split(self.train_prediction_dict[0], self.y_enc):
                    logger.info("Training Level %d Fold # %d. Model # %d", level, foldnum, model_num)

                    if level != 0:
                        l_training_data = temp_train[train_index]
                        l_validation_data = temp_train[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])
                    else:
                        l0_training_data = temp_train[0][model_num]
                        if type(l0_training_data) == list:
                            l_training_data = [x[train_index] for x in l0_training_data]
                            l_validation_data = [x[valid_index] for x in l0_training_data]
                        else:
                            l_training_data = l0_training_data[train_index]
                            l_validation_data = l0_training_data[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])

                    logger.info("Predicting Level %d. Fold # %d. Model # %d", level, foldnum, model_num)

                    if self.task_type == 'classification':
                        temp_train_predictions = model.predict_proba(l_validation_data)
                        self.train_prediction_dict[level][valid_index,
                        (model_num * self.num_classes):(model_num * self.num_classes) +
                                                       self.num_classes] = temp_train_predictions

                    else:
                        temp_train_predictions = model.predict(l_validation_data)
                        self.train_prediction_dict[level][valid_index, model_num] = temp_train_predictions
                    validation_score = self.optimize(self.y_enc[valid_index], temp_train_predictions)
                    validation_scores.append(validation_score)
                    logger.info("Level %d. Fold # %d. Model # %d. Validation Score = %f", level, foldnum, model_num,
                                validation_score)
                    foldnum += 1
                avg_score = np.mean(validation_scores)
                std_score = np.std(validation_scores)
                logger.info("Level %d. Model # %d. Mean Score = %f. Std Dev = %f", level, model_num,
                            avg_score, std_score)

            logger.info("Saving predictions for level # %d", level)
            train_predictions_df = pd.DataFrame(self.train_prediction_dict[level])
            train_predictions_df.to_csv(os.path.join(self.save_path, "train_predictions_level_" + str(level) + ".csv"),
                                        index=False, header=None)

        return self.train_prediction_dict

    def predict(self, test_data, lentest):
        self.test_data = test_data
        if self.task_type == 'classification':
            test_prediction_shape = (lentest, self.num_classes)
        else:
            test_prediction_shape = (lentest, 1)

        self.test_prediction_dict = {}
        for level in range(self.levels):
            self.test_prediction_dict[level] = np.zeros((test_prediction_shape[0],
                                                         test_prediction_shape[1] * len(self.model_dict[level])))
        self.test_data = test_data
        for level in range(self.levels):
            if level == 0:
                temp_train = self.training_data
                temp_test = self.test_data
            else:
                temp_train = self.train_prediction_dict[level - 1]
                temp_test = self.test_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):

                logger.info("Training Fulldata Level %d. Model # %d", level, model_num)
                if level == 0:
                    model.fit(temp_train[0][model_num], self.y_enc)
                else:
                    model.fit(temp_train, self.y_enc)

                logger.info("Predicting Test Level %d. Model # %d", level, model_num)

                if self.task_type == 'classification':
                    if level == 0:
                        temp_test_predictions = model.predict_proba(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict_proba(temp_test)
                    self.test_prediction_dict[level][:, (model_num * self.num_classes): (model_num * self.num_classes) +
                                                                                        self.num_classes] = temp_test_predictions

                else:
                    if level == 0:
                        temp_test_predictions = model.predict(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict(temp_test)
                    self.test_prediction_dict[level][:, model_num] = temp_test_predictions

            test_predictions_df = pd.DataFrame(self.test_prediction_dict[level])
            test_predictions_df.to_csv(os.path.join(self.save_path, "test_predictions_level_" + str(level) + ".csv"),
                                       index=False, header=None)

        return self.test_prediction_dict


In [57]:
# specify the data to be used for every level of ensembling:
train_data_dict = {0: [xtrain_tfv, xtrain_ctv, xtrain_tfv, xtrain_ctv], 1: [xtrain_glove]}
test_data_dict = {0: [xvalid_tfv, xvalid_ctv, xvalid_tfv, xvalid_ctv], 1: [xvalid_glove]}

model_dict = {0: [LogisticRegression(), LogisticRegression(), MultinomialNB(alpha=0.1), MultinomialNB()],

              1: [xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)]}

ens = Ensembler(model_dict=model_dict, num_folds=3, task_type='classification',
                optimize=multiclass_logloss, lower_is_better=True, save_path='')

ens.fit(train_data_dict, ytrain, lentrain=xtrain_glove.shape[0])
preds = ens.predict(test_data_dict, lentest=xvalid_glove.shape[0])

[15:38:54] INFO Found 3 classes
[15:38:54] INFO Training Level 0 Fold # 1. Model # 0
[15:38:54] INFO Predicting Level 0. Fold # 1. Model # 0
[15:38:54] INFO Level 0. Fold # 1. Model # 0. Validation Score = 0.626621
[15:38:54] INFO Training Level 0 Fold # 2. Model # 0


/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[15:38:56] INFO Predicting Level 0. Fold # 2. Model # 0
[15:38:56] INFO Level 0. Fold # 2. Model # 0. Validation Score = 0.616470
[15:38:56] INFO Training Level 0 Fold # 3. Model # 0


/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[15:38:57] INFO Predicting Level 0. Fold # 3. Model # 0
[15:38:57] INFO Level 0. Fold # 3. Model # 0. Validation Score = 0.619626
[15:38:57] INFO Level 0. Model # 0. Mean Score = 0.620906. Std Dev = 0.004242
[15:38:57] INFO Training Level 0 Fold # 1. Model # 1


...

[15:39:10] INFO Predicting Level 0. Fold # 1. Model # 1
[15:39:10] INFO Level 0. Fold # 1. Model # 1. Validation Score = 0.573485
[15:39:10] INFO Training Level 0 Fold # 2. Model # 1
[15:39:24] INFO Predicting Level 0. Fold # 2. Model # 1
[15:39:24] INFO Level 0. Fold # 2. Model # 1. Validation Score = 0.563451
[15:39:24] INFO Training Level 0 Fold # 3. Model # 1
[15:39:38] INFO Predicting Level 0. Fold # 3. Model # 1
[15:39:39] INFO Level 0. Fold # 3. Model # 1. Validation Score = 0.567765
[15:39:39] INFO Level 0. Model # 1. Mean Score = 0.568233. Std Dev = 0.004110
[15:39:39] INFO Training Level 0 Fold # 1. Model # 2
[15:39:39] INFO Predicting Level 0. Fold # 1. Model # 2
[15:39:39] INFO Level 0. Fold # 1. Model # 2. Validation Score = 0.463292
[15:39:39] INFO Training Level 0 Fold # 2. Model # 2
[15:39:39] INFO Predicting Level 0. Fold # 2. Model # 2
[15:39:39] INFO Level 0. Fold # 2. Model # 2. Validation Score = 0.456477
[15:39:39] INFO Training Level 0 Fold # 3. Model # 2
[15:39:

/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[15:40:40] INFO Predicting Test Level 0. Model # 1
[15:40:40] INFO Training Fulldata Level 0. Model # 2
[15:40:40] INFO Predicting Test Level 0. Model # 2
[15:40:40] INFO Training Fulldata Level 0. Model # 3
[15:40:40] INFO Predicting Test Level 0. Model # 3
[15:40:40] INFO Training Fulldata Level 1. Model # 0


/srv/local/data/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[15:40:40] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[15:40:59] INFO Predicting Test Level 1. Model # 0


In [58]:
# check error:
multiclass_logloss(yvalid, preds[1])

0.4717986801266569

Thus, we see that ensembling improves the score by a great extent! Since this is supposed to be a tutorial only I wont be providing any CSVs that you can submit to the leaderboard.

I hope you like it! 

P.S.: If the response is good, I'll add more stuff in this! :)